In [1]:
print("hi")

hi


In [2]:
os.cpu_count()

16

In [2]:
!git clone https://github.com/bigcode-project/selfcodealign.git

fatal: destination path 'selfcodealign' already exists and is not an empty directory.


In [3]:
!cd selfcodealign

In [1]:
%cd selfcodealign/seed_gathering

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/bhargavj20002/code/Users/bhargavj2000/selfcodealign/seed_gathering'

In [6]:
cd ..

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
!git clone https://github.com/tree-sitter/tree-sitter-python.git

Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 3520, done.
remote: Counting objects: 100% (1375/1375), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 3520 (delta 1262), reused 1174 (delta 1174), pack-reused 2145 (from 3)
Receiving objects: 100% (3520/3520), 22.54 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (2148/2148), done.
Updating files: 100% (76/76), done.


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 MB 17.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install boto3

In [6]:
from tree_sitter import Language

Language.build_library(
    'build/my-languages.so',               # output path
    ['tree-sitter-java']                   # list of language repos
)

True

In [ ]:
from tree_sitter import Language, Parser
import datasets
import os
import signal
from multiprocessing import Pool
import boto3
import smart_open
from botocore import UNSIGNED
from botocore.config import Config

# PY_LANGUAGE = Language(tree_sitter_python.language())

# parser = Parser(PY_LANGUAGE)

# Load Java language for Tree-sitter
# JAVA_LANGUAGE = Language(tree_sitter_java.language())
from tree_sitter import Language, Parser

JAVA_LANGUAGE = Language('build/my-languages.so', 'java')
def make_parser():
    parser = Parser(JAVA_LANGUAGE)
    return parser

# S3 access setup
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))
def download_contents(blob_id, src_encoding):
    s3_url = f"s3://softwareheritage/content/{blob_id}"
    with smart_open.open(s3_url, "rb", compression=".gz", transport_params={"client": s3}) as fin:
        content = fin.read().decode(src_encoding)
    return content

# Tree-sitter Java method query
JAVA_METHOD_QUERY = JAVA_LANGUAGE.query("""
(
 (method_declaration
    name: (identifier) @method.name
 ) @method.decl
)
""")

# Helper to extract method declarations from parse tree
def node_to_string(src, node):
    return src[node.start_byte:node.end_byte].decode("utf-8")

def get_java_methods(src, tree):
    captures = JAVA_METHOD_QUERY.captures(tree.root_node)
    res = []
    for node, ty in captures:
        if ty == "method.decl":
            method_code = node_to_string(src, node)
            res.append(method_code)
    return res

def parse_ex(parser, ex):
    ex = download_contents(ex["blob_id"], ex["src_encoding"])
    try:
        buf = bytes(ex, "utf8")
        tree = parser.parse(buf)
        return get_java_methods(buf, tree)
    except:
        return []

PARSERS = None

def process_chunk(idx_and_chunk):
    assert PARSERS is not None
    idx, chunk = idx_and_chunk
    parser = PARSERS[idx]
    chunk_new_methods = set()
    for ex in chunk:
        chunk_new_methods.update(parse_ex(parser, ex))
    return chunk_new_methods


ds = datasets.load_dataset(
"bigcode/the-stack-v2-dedup",
"Java",
split="train",
streaming=False
)


In [ ]:
methods = set()
PARSERS = [make_parser() for _ in range(args.num_workers)]
total_len = len(ds)
CHUNK_SIZE = 1000 * args.num_workers

print(f"Total length: {total_len}")
print(f"Chunk size: {CHUNK_SIZE}")

chunk = []
p = Pool(args.num_workers)
for i, ex in enumerate(ds):
    if i % (total_len // 100) == 0:
        print(f"{i}/{total_len}")
    try:
        chunk.append(ex)
        if len(chunk) == CHUNK_SIZE or i == total_len - 1:
            print(f"Processing chunk {i // CHUNK_SIZE}")
            subchunk_size = len(chunk) // args.num_workers
            subchunks = [chunk[i:i + subchunk_size] for i in range(0, len(chunk), subchunk_size)]
            new_methods_iter = p.imap(process_chunk, [(i, subchunk) for i, subchunk in enumerate(subchunks)])
            print("Getting new methods")
            len_before = len(methods)
            while True:
                try:
                    def timeout_handler(_, __):
                        raise KeyboardInterrupt
                    signal.signal(signal.SIGALRM, timeout_handler)
                    signal.alarm(60)
                    methods.update(next(new_methods_iter))
                    signal.alarm(0)
                except KeyboardInterrupt:
                    signal.alarm(0)
                    print("Keyboard interrupt. Terminating pool")
                    p.terminate()
                    p = Pool(args.num_workers)
                    break
                except StopIteration:
                    break
                except Exception as e:
                    print(e)

            signal.alarm(0)
            PARSERS = [make_parser() for _ in range(args.num_workers)]
            print(f"Done processing chunk {i // CHUNK_SIZE}. Got {len(methods) - len_before} new methods")
            chunk = []
    except Exception as e:
        print(e)
        chunk = []

    if i == total_len - 1:
        break

p.close()

new_ds_dict = {
    "content": list(methods),
    "id": list(range(len(methods)))
}

new_ds = datasets.Dataset.from_dict(new_ds_dict)
# new_ds.push_to_hub(args.push, private=True)


In [7]:
from tree_sitter import Language, Parser
import datasets
import os
import signal
from multiprocessing import Pool
import boto3
import smart_open
from botocore import UNSIGNED
from botocore.config import Config

# Load Java language for Tree-sitter
JAVA_LANGUAGE = Language('build/my-languages.so', 'java')

def make_parser():
    parser = Parser()
    parser.set_language(JAVA_LANGUAGE)
    return parser

# S3 access setup
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))
def download_contents(blob_id, src_encoding):
    s3_url = f"s3://softwareheritage/content/{blob_id}"
    with smart_open.open(s3_url, "rb", compression=".gz", transport_params={"client": s3}) as fin:
        content = fin.read().decode(src_encoding)
    return content

# Tree-sitter Java method query
JAVA_METHOD_QUERY = JAVA_LANGUAGE.query("""
(
 (method_declaration
    name: (identifier) @method.name
 ) @method.decl
)
""")

# Helper to extract method declarations from parse tree
def node_to_string(src, node):
    return src[node.start_byte:node.end_byte].decode("utf-8")

def get_java_methods(src, tree):
    captures = JAVA_METHOD_QUERY.captures(tree.root_node)
    res = []
    for node, ty in captures:
        if ty == "method.decl":
            method_code = node_to_string(src, node)
            res.append(method_code)
    return res

def parse_ex(parser, ex):
    ex = download_contents(ex["blob_id"], ex["src_encoding"])
    try:
        buf = bytes(ex, "utf8")
        tree = parser.parse(buf)
        return get_java_methods(buf, tree)
    except:
        return []

PARSERS = None

def process_chunk(idx_and_chunk):
    assert PARSERS is not None
    idx, chunk = idx_and_chunk
    parser = PARSERS[idx]
    chunk_new_methods = set()
    for ex in chunk:
        chunk_new_methods.update(parse_ex(parser, ex))
    return chunk_new_methods

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def main(args):
    global PARSERS
    ds = datasets.load_dataset(
        args.dataset,
        data_dir=args.data_dir,
        split="train",
    )
    methods = set()
    PARSERS = [make_parser() for _ in range(args.num_workers)]
    total_len = len(ds)
    CHUNK_SIZE = 1000 * args.num_workers

    print(f"Total length: {total_len}")
    print(f"Chunk size: {CHUNK_SIZE}")

    chunk = []
    p = Pool(args.num_workers)
    for i, ex in enumerate(ds):
        if i % (total_len // 100) == 0:
            print(f"{i}/{total_len}")
        try:
            chunk.append(ex)
            if len(chunk) == CHUNK_SIZE or i == total_len - 1:
                print(f"Processing chunk {i // CHUNK_SIZE}")
                subchunk_size = len(chunk) // args.num_workers
                subchunks = [chunk[i:i + subchunk_size] for i in range(0, len(chunk), subchunk_size)]
                new_methods_iter = p.imap(process_chunk, [(i, subchunk) for i, subchunk in enumerate(subchunks)])
                print("Getting new methods")
                len_before = len(methods)
                while True:
                    try:
                        def timeout_handler(_, __):
                            raise KeyboardInterrupt
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(60)
                        methods.update(next(new_methods_iter))
                        signal.alarm(0)
                    except KeyboardInterrupt:
                        signal.alarm(0)
                        print("Keyboard interrupt. Terminating pool")
                        p.terminate()
                        p = Pool(args.num_workers)
                        break
                    except StopIteration:
                        break
                    except Exception as e:
                        print(e)

                signal.alarm(0)
                PARSERS = [make_parser() for _ in range(args.num_workers)]
                print(f"Done processing chunk {i // CHUNK_SIZE}. Got {len(methods) - len_before} new methods")
                chunk = []
        except Exception as e:
            print(e)
            chunk = []

        if i == total_len - 1:
            break

    p.close()

    new_ds_dict = {
        "content": list(methods),
        "id": list(range(len(methods)))
    }

    new_ds = datasets.Dataset.from_dict(new_ds_dict)
    # new_ds.push_to_hub(args.push, private=True)

In [9]:
NUMWORKERS = os.cpu_count()

In [15]:
ds = datasets.load_dataset("bigcode/the-stack-v2-dedup", "Java", cache_dir=f"../nk569/stack", streaming=False, split="train")



In [16]:
funs = set()
PARSERS = [make_parser() for _ in range(NUMWORKERS)]
total_len = len(ds)
CHUNK_SIZE = 1000 * NUMWORKERS

print(f"Total length: {total_len}")
print(f"Chunk size: {CHUNK_SIZE}")

chunk = []
p = Pool(NUMWORKERS)

Total length: 124255350
Chunk size: 16000


Process ForkPoolWorker-6:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-8:


In [17]:
for i, ex in enumerate(iter(ds)):
    if i % (total_len // 100) == 0:
        print(f"{i}/{total_len}")
    try:
        chunk.append(ex)
        if len(chunk) == CHUNK_SIZE or i == total_len - 1:
            print(f"Processing chunk {i // CHUNK_SIZE}")
            # divide the chunk into NUM_WORKERS chunks
            subchunk_size = len(chunk) // NUMWORKERS
            subchunks = [chunk[i:i + subchunk_size]
                         for i in range(0, len(chunk), subchunk_size)]
            new_funs_iter = p.imap(
                process_chunk, [(i, subchunk) for i, subchunk in enumerate(subchunks)])
            print("Getting new functions")
            len_before = len(funs)
            while True:
                try:
                    def timeout_handler(_, __):
                        raise KeyboardInterrupt  # it's fineeeeeee
                    signal.signal(signal.SIGALRM, timeout_handler)
                    signal.alarm(60)
                    funs.update(next(new_funs_iter))
                    signal.alarm(0)
                except KeyboardInterrupt:
                    signal.alarm(0)
                    print("Keyboard interrupt. Terminating pool")
                    p.terminate()
                    p = Pool(NUMWORKERS)
                    break
                except StopIteration:
                    break
                except Exception as e:
                    print(e)

            signal.alarm(0)

            PARSERS = [make_parser() for _ in range(NUMWORKERS)]

            print(
                f"Done processing chunk {i // CHUNK_SIZE}. Got {len(funs) - len_before} new functions")

            chunk = []
    except Exception as e:
        print(e)
        chunk = []

    if i == total_len - 1:
        break


p.close()
new_ds_dict = {
    "content": list(funs),
    "id": list(range(len(funs)))
}

new_ds = datasets.Dataset.from_dict(new_ds_dict)

0/124255350
Processing chunk 0
Getting new functions
Done processing chunk 0. Got 92306 new functions
Processing chunk 1
Getting new functions
Done processing chunk 1. Got 94851 new functions
Processing chunk 2
Getting new functions
unable to access bucket: 'softwareheritage' key: 'content/e7a2fdfea63733b2d75b2183c9862d616121a8b6' version: None error: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
Done processing chunk 2. Got 82534 new functions
Processing chunk 3
Getting new functions
Done processing chunk 3. Got 89784 new functions
Processing chunk 4
Getting new functions
Done processing chunk 4. Got 90459 new functions
Processing chunk 5
Getting new functions
Done processing chunk 5. Got 87630 new functions
Processing chunk 6
Getting new functions
Done processing chunk 6. Got 86135 new functions
Processing chunk 7
Getting new functions
Done processing chunk 7. Got 87283 new functions
Processing chunk 8
Getting new functions
Done

Process ForkPoolWorker-45:
Process ForkPoolWorker-48:
Process ForkPoolWorker-46:
Process ForkPoolWorker-41:
Process ForkPoolWorker-47:
Process ForkPoolWorker-84:
Process ForkPoolWorker-85:
Process ForkPoolWorker-86:
Process ForkPoolWorker-90:
Process ForkPoolWorker-95:
Process ForkPoolWorker-91:
Process ForkPoolWorker-93:
Process ForkPoolWorker-88:
Process ForkPoolWorker-94:
Process ForkPoolWorker-83:
Process ForkPoolWorker-82:
Process ForkPoolWorker-87:
Process ForkPoolWorker-96:
Process ForkPoolWorker-89:
Process ForkPoolWorker-81:
Process ForkPoolWorker-92:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (mo

KeyboardInterrupt: 

In [22]:
new_ds_dict = {
    "content": list(funs),
    "id": list(range(len(funs)))
}

new_ds = datasets.Dataset.from_dict(new_ds_dict)

In [24]:
ds = new_ds

In [30]:
ds

Dataset({
    features: ['content', 'id'],
    num_rows: 180222
})

In [23]:
import subprocess
import tempfile
import signal
import hashlib
import os
import argparse
from typing import List, Dict
from tqdm import tqdm
from tree_sitter_parser import LANGUAGE, global_parser

RETURN_QUERY = LANGUAGE.query("""
(return_statement) @return
""")

def does_have_return(src):
    tree = global_parser.parse(bytes(src, "utf8"))
    root = tree.root_node
    captures = RETURN_QUERY.captures(root)
    for node, _ in captures:
        # if it doesn't have an argument, it's not a return with a value
        if len(node.children) <= 1:  # includes "return" itself
            continue
        else:
            return True
    return False

# runs mypy in the given directory, returns stdout
# then, it logs the number of errors for each file
def run_mypy(d):
    try:
        outs = subprocess.run(
            ["java", "."],
            cwd=d,
            capture_output=True,
            timeout=120,
            text=True,
        ).stdout
    except Exception as e:
        print(e)
        return None

    filemap = {}
    lines = outs.split("\n")
    for line in lines:
        if line.strip():
            parts = line.split(":")
            if len(parts) >= 2:
                file = parts[0].split("/")[-1]
                if file not in filemap:
                    filemap[file] = 0
                if "error:" in line:
                    filemap[file] += 1

    return filemap

def typecheck_batch(files: List[str]) -> Dict[str, str]:
    # Create a temporary directory using the tempfile module
    filemap: Dict[str, str] = {}
    with tempfile.TemporaryDirectory() as tempdir:
        for contents in files:
            hash_object = hashlib.sha1(bytes(contents, "utf8"))
            hex_dig = hash_object.hexdigest()
            filemap[hex_dig] = contents
            name = os.path.join(tempdir, hex_dig + ".py")
            with open(name, "w") as f:
                f.write(contents)

        # Run mypy in the temporary directory
        typecheck_map = run_mypy(tempdir)
        print(typecheck_map)

        if typecheck_map is None:
            return {}

        for contents, errors in typecheck_map.items():
            no_py = contents.replace(".py", "")
            if errors == 0:
                continue
            if no_py in filemap:
                del filemap[no_py]

        print(f"Pass rate: {len(filemap)}/{len(files)}")
        return filemap

def infer_imports(code: str) -> str:
    import autoimport
    try:
        def handler(signum, frame):
            raise Exception("Timeout")
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(10)
        inferred = autoimport.fix_code(code)
        signal.alarm(0)
        return inferred
    except Exception as e:
        signal.alarm(0)
        print(f"Error while inferring imports: {e}")
        return code

In [26]:
print("Filtering to only functions with return statements")
ds = ds.filter(lambda ex: does_have_return(
    ex["content"]), num_proc=os.cpu_count())

Filtering to only functions with return statements


Filter (num_proc=16): 100%|██████████| 1152832/1152832 [02:36<00:00, 7385.88 examples/s] 


In [31]:
ds

{'content': 'public java.lang.Integer getNumPontoDepol(){\n        return this.numPontoDepol;\n    }',
 'id': 0}

In [32]:
def run_javac(directory: str) -> Dict[str, int]:
    try:
        result = subprocess.run(
            ["javac", "*.java"],
            cwd=directory,
            shell=True,
            capture_output=True,
            timeout=120,
            text=True,
        )
        output = result.stderr  # `javac` prints errors to stderr
    except Exception as e:
        print(f"Compilation error: {e}")
        return None

    filemap = {}
    for line in output.splitlines():
        if ".java:" in line:
            file = line.split(":")[0].split("/")[-1]
            filemap[file] = filemap.get(file, 0) + 1
    return filemap

# --- Filter batch of Java snippets based on successful compilation ---
def typecheck_batch(files: List[str]) -> Dict[str, str]:
    filemap: Dict[str, str] = {}
    with tempfile.TemporaryDirectory() as tempdir:
        for contents in files:
            hexname = hashlib.sha1(bytes(contents, "utf8")).hexdigest()
            filemap[hexname] = contents
            file_path = os.path.join(tempdir, hexname + ".java")
            with open(file_path, "w") as f:
                f.write(contents)

        compile_results = run_javac(tempdir)
        print("Compilation results:", compile_results)

        if compile_results is None:
            return {}

        for filename, error_count in compile_results.items():
            hexname = filename.replace(".java", "")
            if error_count > 0 and hexname in filemap:
                del filemap[hexname]

        print(f"✅ Pass rate: {len(filemap)}/{len(files)}")
        return filemap

# --- Placeholder for Java imports (not implemented) ---
def infer_imports(code: str) -> str:
    # Java requires manual import handling; consider using a parser or model to infer
    return code

In [33]:
batch = []
max_i = len(ds) - 1

new_ds = {
    "content": [],
    "sha1": [],
    "id": [],
}

e_id = 0

for i, ex in enumerate(tqdm(ds, total=len(ds))):
    try:
        code = ex["content"]

        batch.append(code)

        if len(batch) == 250 or i == max_i:
            filemap = typecheck_batch(batch)
            for sha1, contents in filemap.items():
                new_ds["content"].append(contents)
                new_ds["sha1"].append(sha1)
                new_ds["id"].append(e_id)
                e_id += 1
            batch = []
            
    except Exception as e:
        print(f"There was an error: {e}")
        continue

new_ds_hf = datasets.Dataset.from_dict(new_ds)

100%|██████████| 180222/180222 [02:53<00:00, 1041.46it/s]


Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilation results: {}
✅ Pass rate: 250/250
Compilatio

In [34]:
print(new_ds_hf['content'][0])

public java.lang.Integer getNumPontoDepol(){
        return this.numPontoDepol;
    }


In [35]:
save_dir = "../datasets/seed2"

In [2]:
!zip -r seed2.zip ../datasets/seed2/

  adding: ../datasets/seed2/ (stored 0%)
  adding: ../datasets/seed2/.amlignore (deflated 32%)
  adding: ../datasets/seed2/.amlignore.amltmp (deflated 32%)
  adding: ../datasets/seed2/data-00000-of-00001.arrow (deflated 67%)
  adding: ../datasets/seed2/dataset_info.json (deflated 58%)
  adding: ../datasets/seed2/state.json (deflated 38%)


In [36]:
new_ds_hf.save_to_disk(save_dir)

Saving the dataset (1/1 shards): 100%|██████████| 180222/180222 [00:00<00:00, 262970.97 examples/s]


### SEED GATHERING FILTER DATASET

In [37]:
import datasets
import os
from tree_sitter_parser import global_parser, LANGUAGE, does_have_return, make_parser
import benchmark_data
from tqdm import tqdm
import torch
import argparse
from vllm import LLM, SamplingParams
import random

In [39]:

FN_BLOCK_QUERY = LANGUAGE.query("""
(function_definition
  body: (block) @fn-block)
""")

def template_few_shot_java(code, comment, answer, rationale):
    assert answer in {"Yes", "No"}
    prompt = f"""<issue_start>username_0: I have a method in Java and I'd like someone to check my comment describing this method.
I'm doing this so that I can write better documentation for the method.

Here is the code for the method:
```java
{code}
```

Here is my comment describing what it does:
```
{comment}
```

Do not attempt to execute the method or judge its correctness.
Answer with "Yes" or "No" depending on whether the comment provides enough information to re-implement the method.
Also, answer with "No" if the comment does not match the method.<issue_comment>username_1: Sure, no problem. I will be able to help.
My answer is: {answer}

{rationale}

Upvotes: 200"""
    return prompt

FEW_SHOTS_JAVA = [
    (
        "public static double inchesToCentimeters(double inches) {\n    return inches * 2.54;\n}",
        "Converts inches to centimeters.",
        "Yes",
        "The comment clearly and concisely describes the function’s behavior: it multiplies inches by 2.54 to convert to centimeters.",
    ),
    (
        "public static int max(int a, int b) {\n    return (a > b) ? a : b;\n}",
        "Returns the smaller of two integers.",
        "No",
        "The comment is incorrect; the method actually returns the larger (maximum) of the two integers, not the smaller.",
    ),
    (
        "public static int[] reverse(int[] arr) {\n    int[] reversed = new int[arr.length];\n    for (int i = 0; i < arr.length; i++) {\n        reversed[i] = arr[arr.length - 1 - i];\n    }\n    return reversed;\n}",
        "Reverses an array of integers.",
        "Yes",
        "The method creates a new array and fills it by reversing the order of the original array elements. The description is accurate.",
    ),
    (
        "public static void printHelloWorld() {\n    System.out.println(\"Hello, World!\");\n}",
        "Prints a user's name to the console.",
        "No",
        "The comment is incorrect; the method prints 'Hello, World!' not a user’s name.",
    ),
]

def prompt_fmt_java(code, comment):
    import random
    random.shuffle(FEW_SHOTS_JAVA)
    buf = ""
    for few in FEW_SHOTS_JAVA:
        buf += template_few_shot_java(*few)
    buf += f"""<issue_start>username_0: I have a method in Java and I'd like someone to check my comment describing this method.
I'm doing this so that I can write better documentation for the method.

Here is the code for the method:
```java
{code}
```

Here is my comment describing what it does:
```
{comment}
```

Do not attempt to execute the method or judge its correctness.
Answer with "Yes" or "No" depending on whether the comment provides enough information to re-implement the method.
Also, answer with "No" if the comment does not match the method.
Upvotes: 100<issue_comment>username_1: Sure, no problem. I will be able to help.
My answer is:"""
    return buf

def auto_dtype():
    if torch.cuda.is_bf16_supported():
        return "bfloat16"
    return "auto"


def chunkify(lst, n):
    chunks = []
    for i in range(0, len(lst), n):
        chunk = []
        for j in range(n):
            if i + j < len(lst):
                chunk.append(lst[i + j])
        chunks.append(chunk)
    return chunks

In [40]:
dataset = new_ds_hf

In [41]:
len(dataset)

180222

In [44]:
print(f"Loaded {len(dataset)} examples. Running pre-filtering...")

BAD_WORDS = ["todo", "fixme", "bug"]
BAD_IMPORTS = ["java.io", "java.lang.Runtime", "java.nio.file", "java.awt", 
               "javax.swing", "java.lang.reflect"]
BAD_IMPORTS = [f"import {b}" for b in BAD_IMPORTS]
BAD_SUBSTRINGS = BAD_WORDS + BAD_IMPORTS

bench_filter = benchmark_data.filter_out()
all_bench = bench_filter["human_eval_docstrings"] + \
    bench_filter["human_eval_solutions"] + \
    bench_filter["mbpp_docstrings"] + \
    bench_filter["mbpp_solutions"]

Loaded 180222 examples. Running pre-filtering...
num strings from mbpp_docstrings: 120
num strings from mbpp_solutions: 120
num strings from human_eval_docstrings: 164
num strings from human_eval_solutions: 161


In [ ]:
def pre_filtering(ex):
    code = ex["content"]
    code_bytes = code.encode('utf-8')

    # filter out bad substrings
    lower = code.lower()
    for word in BAD_SUBSTRINGS:
        if word in lower:
            return False

    for b in all_bench:
        if b in code:  # contaminated sample!
            return False

    # too many lines of code -- say 150
    lines = code.split("\n")
    if len(lines) > 150:
        return False

    # filter out functions with no return statement
    parser = make_parser()
    if not does_have_return(code, parser=parser):
        return False


    return True  # all good!


threads = os.cpu_count() - 1  # type: ignore
dataset2 = dataset.filter(pre_filtering, num_proc=threads)

In [48]:
dataset

In [ ]:
model = LLM(f"../../../StarCoder", dtype=auto_dtype(),
            gpu_memory_utilization=0.95, tensor_parallel_size=1)


In [ ]:
tokenizer = model.get_tokenizer()

In [ ]:
print(f"Now running stage 3 filtering on {len(dataset)} examples...")

Now running stage 3 filtering on 236926 examples...


In [ ]:
def unindent(s):
    lines = s.splitlines()
    non_blank_lines = [line for line in lines if line.strip()]
    min_indent = min(len(line) - len(line.lstrip())
                     for line in non_blank_lines) if non_blank_lines else 0
    unindented_lines = [line[min_indent:] if len(
        line) >= min_indent else line for line in lines]
    return '\n'.join(unindented_lines)


def py_extract_docstring(code):
    first_doc = code.find('"""')
    assert first_doc != -1
    first_doc = first_doc + 3
    second_doc = code[first_doc+1:].find('"""')
    assert second_doc != -1
    second_doc = second_doc + first_doc + 1
    doc = code[first_doc:second_doc]
    doc = unindent(doc).strip()
    code = code[:first_doc-3] + code[second_doc+3:]
    return doc, code



In [ ]:
dummy = 'def dummy(): \n    """\n    """\n pass'
dummy_prompt = prompt_fmt(dummy)
few_shot_toks = len(tokenizer.encode(
    dummy_prompt)) - len(tokenizer.encode(dummy))
print(f"Few-shot prompt has {few_shot_toks} tokens")

Few-shot prompt has 2573 tokens


In [ ]:
prompts = []
for ex in tqdm(dataset, total=len(dataset), desc="Generating prompts"):
    code = ex["content"]
    toks = len(tokenizer.encode(code)) + few_shot_toks
    if toks > 16380:
        print(f"Skipping example with {toks} tokens")
        # to skip, just add dummy prompt
        prompts.append(dummy_prompt)
        continue
    p = prompt_fmt(code)
    prompts.append(p)

responses = []
for chunk in tqdm(chunkify(prompts, 512), desc="Generating responses"):
    outs = model.generate(chunk, SamplingParams(
        temperature=0.0, stop="\n", max_tokens=5))
    contents = [o.outputs[0].text for o in outs]
    for c in contents:
        yes_count = c.lower().count("yes")
        no_count = c.lower().count("no")
        if yes_count > no_count:
            responses.append(True)
        elif yes_count < no_count:
            responses.append(False)
        else:
            # default to No
            responses.append(False)



Generating prompts:  15%|██████████████████████▉                                                                                                                                  | 35479/236926 [00:21<02:06, 1588.67it/s]

Skipping example with 29237 tokens


Generating prompts:  35%|█████████████████████████████████████████████████████▊                                                                                                   | 83335/236926 [00:49<01:36, 1595.44it/s]

Skipping example with 20286 tokens


Processed prompts:   0%|                                                                                                                       | 0/512 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
dataset

Dataset({
    features: ['content', 'sha1', 'id'],
    num_rows: 236926
})

In [ ]:
subset = dataset.select(range(75000))

In [ ]:
subset

Dataset({
    features: ['content', 'sha1', 'id'],
    num_rows: 75000
})

In [ ]:
new_ds = subset.filter(  # horrible hack!
    lambda ex, i: responses[i] and "def dummy()" not in ex["content"], with_indices=True)
print(f"Filtered {len(dataset) - len(new_ds)} examples")

Filter:   0%|          | 0/75000 [00:00<?, ? examples/s]

Filtered 233319 examples


In [ ]:
new_ds.save_to_disk("../datasets/seed3")

Saving the dataset (0/1 shards):   0%|          | 0/3607 [00:00<?, ? examples/s]

In [ ]:
new_ds

Dataset({
    features: ['content', 'sha1', 'id'],
    num_rows: 3607
})